In [2]:
from urllib.request import urlopen
import numpy as np
import cv2
import time
import torch
from ultralytics import YOLO
import socket

server_ip = "192.168.186.173"  # NodeMCU IP
server_port = 1000  # NodeMCU 포트번호
socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#TCP/IP 통신을 위한 소켓 생성
socket.connect((server_ip, server_port)) #연결

model = YOLO("best_run21_x.pt")

url = 'http://192.168.186.11/cam-hi.jpg'

img_cnt = 0

winName = 'ESP32 CAMERA'

cv2.namedWindow("쓰레기 객체인식",cv2.WINDOW_AUTOSIZE)
prev_time = 0
while True:
    imgResponse = urlopen(url)
    imgNp = np.array(bytearray(imgResponse.read()),dtype=np.uint8)
    img = cv2.imdecode (imgNp,-1) 
            
    results = model.predict(img, conf=0.75, verbose=False)  # predict on an image
    plots=results[0].plot()
    detected_num = len(results[0].boxes.cls)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    if detected_num == 1 and time.time() - prev_time >= 10:
        class_num = int(results[0].boxes.cls[torch.argmax(results[0].boxes.conf)].item())
        class_name = results[0].names[class_num]
        print(results[0].names[class_num])
        
        msg = class_name #판독한 객체 정보를 msg에 저장
        socket.sendall(msg.encode(encoding="utf-8")) #msg을 utf-8 인코딩하여 송신
        
        cv2.imwrite(f'saved_images/plot{img_cnt}_{class_name}.jpg', plots)
        img_cnt += 1
        prev_time = time.time()
        
    cv2.imshow("쓰레기 객체인식", plots)
        
        
cv2.destroyAllWindows()